# Predicting Stock Prices from Expected Earnings
## 1. Overview
This project uses a Long Short-Term Memory (LSTM) network through TensorFlow to predict stock prices the day after an earnings report. It takes as input the daily stock prices for the (approximately) 3-month period leading up to the earnings report, as well as the expected Earnings Per Share (EPS) prior to the report. Its output is the expected % difference between the next day's closing price and the previous day's high price.
## 2. Collecting and Cleaning Data
Data will be taken from the last 8 years' historical stock prices and earnings reports. It is housed in stocks_latest, which you should download on your own at <a href="https://www.kaggle.com/tsaustin/us-historical-stock-prices-with-earnings-data">this link</a>.

First, let's import the packages we'll be using.

In [1]:
import os
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import seaborn as sns
import tensorflow as tf
from tensorflow.python.client import device_lib
from IPython.display import display, clear_output
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

Make sure we're running tensorflow on GPU (if applicable):

In [2]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1



Next, we'll extract our data as pandas dataframes.

In [3]:
stocks_data_path = "./datasets/stock_prices_latest.csv"
earnings_data_path = "./datasets/earnings_latest.csv"

stocks_df_init = pd.read_csv(stocks_data_path)
earnings_df_init = pd.read_csv(earnings_data_path)

Let's see what data our stocks and earnings files contain:

In [4]:
stocks_df = stocks_df_init.copy()
stocks_df.head()

,symbol,date,open,high,low,close,close_adjusted,volume,split_coefficient
0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,49.7013,20032017,1.0
1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,22.5902,30977700,1.0
2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,18.0802,41591300,1.0
3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,27.2232,288121200,1.0
4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,38.6773,74640000,1.0


In [5]:
earnings_df = earnings_df_init.copy()
earnings_df.head()

,symbol,date,qtr,eps_est,eps,release_time
0,A,2009-05-14,04/2009,NaN,NaN,post
1,A,2009-08-17,07/2009,NaN,NaN,post
2,A,2009-11-13,10/2009,NaN,NaN,pre
3,A,2010-02-12,01/2010,NaN,NaN,pre
4,A,2010-05-17,04/2010,NaN,NaN,post


We'll do the following to clean up the stocks dataset:
1. Remove split_coefficient and close, opting for close_adjusted instead
2. Rename close_adjusted to close
3. Convert our dates to datetime objects in pandas
4. Sort first by symbol, then by date
5. Remove all data from before 2015

In [6]:
stocks_df.drop(['split_coefficient','close'], axis=1, errors='ignore', inplace=True)
stocks_df['date'] = pd.to_datetime(stocks_df.date)
stocks_df['percent_change'] = (stocks_df['open'] - stocks_df['high'].shift(-1)) / stocks_df['high'].shift(-1)
stocks_df = stocks_df[(stocks_df['date'].dt.year >= 2015)]
stocks_df.sort_values(by=['symbol','date'], inplace=True)
stocks_df.head()

,symbol,date,open,high,low,close_adjusted,volume,percent_change
8237673,A,2015-01-02,41.18,41.31,40.37,39.3484,1530798,0.004390
8236049,A,2015-01-05,40.32,40.46,39.70,38.6111,2042240,0.176196
8234683,A,2015-01-06,39.81,40.02,39.02,38.0096,2084562,-0.384318
8237067,A,2015-01-07,39.52,39.81,39.29,38.5141,3359660,-0.587818
8238420,A,2015-01-08,40.24,40.98,40.18,39.6686,2116341,1.857955


We'll be feeding our NN the stock data for individual stocks. First, we'll group the stock data by symbol and store in a dict of DataFrames.

In [10]:
stocks_sep = dict(tuple(stocks_df.groupby('symbol')))
symbol1 = stocks_sep['ZM']
stocks_sep['ZM'].head()

,symbol,date,open,high,low,close_adjusted,volume,percent_change
20646711,ZM,2019-04-18,65.00,66.000,60.321,62.0,25764659,-0.056604
20646712,ZM,2019-04-22,61.00,68.900,59.940,65.7,9949738,-0.098300
20646710,ZM,2019-04-23,66.87,74.169,65.550,69.0,6786513,0.013182
20646715,ZM,2019-04-24,71.40,71.500,63.160,63.2,4973529,5.076596
20646709,ZM,2019-04-25,64.74,66.850,62.600,65.0,3863275,-0.127129


Now we'll need to use our earnings data, and particularly the dates of earnings periods for each company, to further separate our data into 90-day chunks. Let's clean and explore our earnings data:

In [8]:
earnings_df.drop(['release_time', 'qtr'], axis=1, errors='ignore', inplace=True)
earnings_df['eps_est'].mask(pd.isna, earnings_df['eps'], inplace=True)
earnings_df.dropna(inplace=True)
earnings_df['date'] = pd.to_datetime(earnings_df.date)
earnings_df = earnings_df[(earnings_df['date'].dt.year >= 2016)]
earnings_df.sort_values(by=['symbol', 'date'], inplace=True)

We'll need the start and end dates for each earnings period to split our input data. We'll also need each stock's percent change for each period. Let's calculate them and assign them to new columns in earnings_df.

In [9]:
earnings_df['earnings_start'] = earnings_df['date'] - BDay(57)
earnings_df['earnings_end'] = earnings_df['date'] - BDay(1)
earnings_df.head()
print(earnings_df[earnings_df['symbol'] == 'AAON'])

    symbol       date  eps_est   eps earnings_start earnings_end
291   AAON 2016-02-25    0.200  0.24     2015-12-08   2016-02-24
292   AAON 2016-05-05    0.200  0.20     2016-02-16   2016-05-04
293   AAON 2016-08-04    0.270  0.27     2016-05-17   2016-08-03
294   AAON 2016-11-03    0.290  0.29     2016-08-16   2016-11-02
295   AAON 2017-02-23    0.240  0.21     2016-12-06   2017-02-22
296   AAON 2017-05-04    0.200  0.19     2017-02-14   2017-05-03
297   AAON 2017-08-03    0.280  0.26     2017-05-16   2017-08-02
298   AAON 2017-11-02    0.300  0.28     2017-08-15   2017-11-01
299   AAON 2018-02-27    0.240  0.22     2017-12-08   2018-02-26
300   AAON 2018-05-03    0.240  0.08     2018-02-13   2018-05-02
301   AAON 2018-08-02    0.200  0.22     2018-05-15   2018-08-01
302   AAON 2018-11-01    0.300  0.27     2018-08-14   2018-10-31
303   AAON 2019-02-28    0.277  0.24     2018-12-11   2019-02-27
304   AAON 2019-05-02    0.233  0.21     2019-02-12   2019-05-01
305   AAON 2019-10-31    

In [10]:
earnings_sep = dict(tuple(earnings_df.groupby('symbol')))
delete_dict = []
for symbol in earnings_sep:
    try:
        stocks_sep[symbol]
    except:
        delete_dict.append(symbol)
for symbol in delete_dict:
    del earnings_sep[symbol]
symbol2 = earnings_sep['ZM']
symbol2.head()

,symbol,date,eps_est,eps,earnings_start,earnings_end
160388,ZM,2019-06-06,0.005,0.03,2019-03-19,2019-06-05
160389,ZM,2019-09-05,0.013,0.08,2019-06-18,2019-09-04
160390,ZM,2019-12-05,0.028,0.09,2019-09-17,2019-12-04
160391,ZM,2020-03-04,0.071,0.15,2019-12-16,2020-03-03
160392,ZM,2020-06-02,0.092,0.20,2020-03-13,2020-06-01


We'll now iterate through our earnings and generate input period and output percent increases for our RNN.

_Note: this is pretty slow and could probably be improved with .apply() or with vectorization._

In [12]:
def normalize_stock_data(df):
    # normalize all stock prices
    arr = np.array(df.to_numpy())
    #stock_high = np.max(arr)
    #arr = arr / stock_high
    # put most recent dates first
    arr = np.flip(arr, axis=0)
    # pad bottom of array with zeros so all are of shape (57, 4)
    num_rows = arr.shape[0]
    arr = np.pad(arr, ((0, 57-num_rows), (0,0)), mode='constant', constant_values = 0)
    return arr

stocks_cnt = 0
stocks_total_cnt = len(earnings_sep)
input_data = []
output_data = []
#earnings_sep_test = {k: earnings_sep[k] for k in list(earnings_sep)[:100]}
for symbol in earnings_sep:
    curr_stock = stocks_sep[symbol].copy()
    stocks_cnt += 1
    if stocks_cnt % 100 == 0 or stocks_cnt == stocks_total_cnt:
        clear_output(wait=True)
        display('stocks processed: '+str(stocks_cnt)+'/'+str(stocks_total_cnt))
    for earnings_row in earnings_sep[symbol].itertuples():
        # compute input data
        stock_period_data = curr_stock[(curr_stock.date >= earnings_row.earnings_start) & (curr_stock.date <= earnings_row.earnings_end)]
        if not stock_period_data.empty:
            stock_period_data.drop(['symbol', 'date', 'volume', 'open', 'high', 'low', 'close_adjusted'], axis=1, errors='ignore', inplace=True)
            stock_period_data = normalize_stock_data(stock_period_data)

            # compute output data
            earnings_prev_high = curr_stock[curr_stock.date == earnings_row.date].high.to_numpy()
            earnings_next_open = curr_stock[curr_stock.date == earnings_row.date + BDay(1)].open.to_numpy()
            percent_change = (earnings_next_open - earnings_prev_high) / earnings_prev_high
            
            if percent_change and -20 <= percent_change <= 20:
                # convert from array to float
                percent_change = percent_change[0]
                
                # store data
                input_data.append(stock_period_data)
                output_data.append(percent_change)

'stocks processed: 5108/5108'

Next we'll shuffle our data and separate it into training (64%), dev (16%) and test (20%) sets.

In [13]:
def get_breakpoint(x, percent):
    return int(np.floor(len(x) * percent))

'''
def categorize_output(y):
    if y > 0.1:
        return np.array([1, 0, 0, 0, 0])
    if y >= 0.05:
        return np.array([0, 1, 0, 0, 0])
    if y >= 0:
        return np.array([0, 0, 1, 0, 0])
    if y >= -0.05:
        return np.array([0, 0, 0, 1, 0])
    else:
        return np.array([0, 0, 0, 0, 1])
'''
def categorize_output(y):
    if y >= 0:
        return np.array([1, 0])
    return np.array([0, 1])

cnt = np.array([0,0])
x, y = shuffle(input_data, output_data)
for i in range(len(y)):
    one_hot = categorize_output(y[i])
    y[i] = one_hot
    cnt += one_hot
print(cnt/len(y))
x = np.array(x)
y = np.array(y)
x = x.reshape(len(x), x.shape[1], 1)
y = y.reshape(len(y), 2)
x = np.float32(x)

breakpoint_1 = get_breakpoint(x, 0.64)
breakpoint_2 = get_breakpoint(x, 0.8)

x_train = x[:breakpoint_1]
y_train = y[:breakpoint_1]
x_dev = x[breakpoint_1+1:breakpoint_2]
y_dev = y[breakpoint_1+1:breakpoint_2]
x_test = x[breakpoint_2+1:]
y_test = y[breakpoint_2+1:]

[0.220798 0.779202]


## 3. Constructing the NN Model
We'll now construct our neural network model:

In [29]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.Conv2D(32, (2,2), padding='same', activation=tf.nn.relu, input_shape=(57, 1, 1)),
    #tf.keras.layers.Conv2D(64, (2,2), padding='same', activation=tf.nn.relu),
    tf.keras.layers.Conv1D(64, 2, activation=tf.nn.relu, input_shape=(57,1), kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None)),
    #tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    #tf.keras.layers.LSTM(32, return_sequences=True),
    tf.keras.layers.Conv1D(32, 2, activation=tf.nn.relu, input_shape=(57,1), kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation=tf.nn.relu, kernel_initializer=tf.keras.initializers.GlorotNormal(seed=None)),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

#optimizer = tf.keras.optimizers.SGD(lr=1e-5, momentum=0.5)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
#loss = tf.keras.losses.Huber()
#loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True, label_smoothing=0.2)
model.compile(loss=loss,
              optimizer=optimizer,
              metrics=["accuracy"])

Now we fit the model with our training data:

In [28]:
model.fit(x_train, y_train, batch_size=256, epochs=5, verbose=1)

Train on 48681 samples
Epoch 1/5
48681/48681 [==============================] - 1s 23us/sample - loss: 0.6586 - accuracy: 0.7773
Epoch 2/5
48681/48681 [==============================] - 1s 11us/sample - loss: 0.6419 - accuracy: 0.7793
Epoch 3/5
48681/48681 [==============================] - 1s 11us/sample - loss: 0.6416 - accuracy: 0.7793
Epoch 4/5
48681/48681 [==============================] - 1s 11us/sample - loss: 0.6416 - accuracy: 0.7793
Epoch 5/5
48681/48681 [==============================] - 1s 11us/sample - loss: 0.6415 - accuracy: 0.7793


In [18]:
test_loss, test_accuracy = model.evaluate(x_dev, y_dev)
print('Accuracy on test dataset:', test_accuracy)

12170/12170 [==============================] - 1s 51us/sample - loss: 0.6832 - accuracy: 0.7800
Accuracy on test dataset: 0.7799507


In [19]:
model.predict(x_dev)

array([[1.7482193e-07, 9.9999988e-01],
       [1.7482193e-07, 9.9999988e-01],
       [1.7482193e-07, 9.9999988e-01],
       ...,
       [1.7482193e-07, 9.9999988e-01],
       [1.7482193e-07, 9.9999988e-01],
       [1.7482193e-07, 9.9999988e-01]], dtype=float32)

The model simply predicts the most common label (that is, the stock decreases) for all test cases. Since the test data decreases 68% of the time, the model claims 68% accuracy, but hasn't learned anything. Next attempt: LSTM with windows.